In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.layers import Dense, Activation, Flatten, Convolution2D, Dropout
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import os
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv(r'C:\Users\rana ahmed\Desktop\Senior_2\deepLearingproject\Leaf-Classification\leaf-classification\train.csv')
test = pd.read_csv(r'C:\Users\rana ahmed\Desktop\Senior_2\deepLearingproject\Leaf-Classification\leaf-classification\test.csv')

In [3]:
# Preparing classes for labeling
traindata = train[['id','species']].copy()
traindata['id'] = traindata['id'].astype(str)
traindata['label'] = LabelEncoder().fit_transform(traindata['species'])


# Loading Images and Linking Labels
images = r'C:\Users\rana ahmed\Desktop\Senior_2\deepLearingproject\Leaf-Classification\leaf-classification\images'

# List comprehension to load images and link labels
imgs = [Image.open(os.path.join(images, i)).convert('1') 
        for i in sorted(os.listdir(images)) 
        if i.split('.')[0] in traindata['id'].values]

# Matching labels using list comprehension
labels = [traindata[traindata['id'] == i.split('.')[0]]['label'].values[0] 
          for i in sorted(os.listdir(images)) 
          if i.split('.')[0] in traindata['id'].values]

# Displaying information
print(f"We have {len(imgs)} images in the dataset")
print(f"Dataset have {len(np.unique(labels))} labels ")

We have 990 images in the dataset
Dataset have 99 labels 


In [4]:
def load_images_as_arrays(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Filter by file extensions
            img = Image.open(os.path.join(directory, filename))
            img_array = np.array(img)
            images.append(img_array)
    return images

images_as_array = load_images_as_arrays(images)
print(len(images_as_array))  # Check the number of loaded images
print(images_as_array)

1584
[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0,

In [5]:
labels_np = np.array(labels)
labels_np

array([ 3, 54, 20,  1, 45,  3,  9, 13,  2, 66, 59, 62, 79,  6, 17, 43, 83,
       26,  1, 12, 49, 71, 89, 58, 88, 93, 39, 42, 15, 38, 55, 15, 93,  4,
       90, 88, 55, 40, 55, 17, 34, 94, 34, 57, 92, 81, 26, 92, 60, 89, 49,
       89, 79, 30, 65, 58,  4, 19,  4, 76, 82, 74, 71, 21, 54, 13, 16, 72,
       78, 68, 62, 61, 25, 72,  7, 12, 18, 77, 90, 62, 14,  3, 78, 65, 37,
       27, 50, 95, 98, 60, 72, 58, 38, 87, 20, 93, 19,  7, 83, 50,  3, 32,
       91, 77,  7, 64, 61, 69, 23, 76, 65, 48, 41, 92, 20, 91, 18,  4, 70,
        9,  9, 29, 85, 67,  0, 35, 98, 84, 91, 90, 31, 53, 39, 24, 85, 96,
       17,  7, 11, 96, 39, 36, 56, 90, 79, 45, 64, 97, 41, 19, 74, 11, 10,
       62, 95, 28, 35, 96, 10,  7, 68,  7, 93, 34, 42, 68, 41, 14, 22, 58,
       12, 71, 27, 98, 72, 91, 72,  3, 43, 19, 61, 75, 20, 81, 63, 67, 56,
       26, 47, 11, 31, 60, 57, 62, 66, 19, 75, 71, 97, 94, 13, 75, 95, 32,
       50, 97, 52, 87, 32, 72,  3, 47, 77, 48, 33, 73, 64, 49, 68, 52, 43,
       94, 77, 68, 50, 47

In [6]:
def resize_images_and_greyscale(images_array, new_size):
    resized_images = []
    for img in images_array:
        pil_img = Image.fromarray(img)
        # Resize
        resized_img = pil_img.resize(new_size)
        # Convert to grayscale
        grayscale_img = resized_img.convert('L')
        resized_images.append(np.array(grayscale_img))
    return resized_images

# Resize images in the array to a new size (e.g., (64, 64)) and convert to greyscale
resized_images = resize_images_and_greyscale(images_as_array, (32, 32))

# Check the shape of the resized and grayscale images
print("Shape of the first image:", resized_images[0].shape)

Shape of the first image: (32, 32)


In [7]:
resized_images = np.expand_dims(resized_images, axis=-1)
print("Shape of the first image:", resized_images[0].shape)

Shape of the first image: (32, 32, 1)
